In [13]:
from glob import glob

paths = glob("../../persist/blob/channels/*/video_snippet/*/*")
paths

['../../persist/blob/channels/UCpVm7bg6pXKo1Pr6k5kxG9A/video_snippet/_OGECa4jFME/comments-pt-1.json',
 '../../persist/blob/channels/UCpVm7bg6pXKo1Pr6k5kxG9A/video_snippet/_OGECa4jFME/comments-pt-2.json',
 '../../persist/blob/channels/UCpVm7bg6pXKo1Pr6k5kxG9A/video_snippet/_OGECa4jFME/comments-pt-0.json',
 '../../persist/blob/channels/UCpVm7bg6pXKo1Pr6k5kxG9A/video_snippet/SF_5jof0GHY/comments-pt-3.json',
 '../../persist/blob/channels/UCpVm7bg6pXKo1Pr6k5kxG9A/video_snippet/SF_5jof0GHY/comments-pt-1.json',
 '../../persist/blob/channels/UCpVm7bg6pXKo1Pr6k5kxG9A/video_snippet/SF_5jof0GHY/comments-pt-2.json',
 '../../persist/blob/channels/UCpVm7bg6pXKo1Pr6k5kxG9A/video_snippet/SF_5jof0GHY/comments-pt-0.json',
 '../../persist/blob/channels/UCpVm7bg6pXKo1Pr6k5kxG9A/video_snippet/-FMvbefokq8/comments-pt-1.json',
 '../../persist/blob/channels/UCpVm7bg6pXKo1Pr6k5kxG9A/video_snippet/-FMvbefokq8/comments-pt-2.json',
 '../../persist/blob/channels/UCpVm7bg6pXKo1Pr6k5kxG9A/video_snippet/-FMvbefokq8/c

In [20]:
import pandas as pd
import json

f = lambda x: x if x else {"comments": []}
comment_list = []
for elem in paths:
    with open(elem) as js_io:
        snp_dct = json.load(js_io)
    comments = [f(x.get("replies")) | x["snippet"] for x in snp_dct["items"]]
    for elem in comments:
        elem["comments"].append(-1)
    comment_list.extend(comments)

In [21]:
df = pd.DataFrame(comment_list)
df

,comments,videoId,topLevelComment,canReply,totalReplyCount,isPublic
0,[-1],_OGECa4jFME,"{'kind': 'youtube#comment', 'etag': 'YuWkwE1fD...",True,0,True
1,[-1],_OGECa4jFME,"{'kind': 'youtube#comment', 'etag': '6uUdKftsY...",True,0,True
2,[-1],_OGECa4jFME,"{'kind': 'youtube#comment', 'etag': 'F3cSycXbJ...",True,0,True
3,"[{'kind': 'youtube#comment', 'etag': 'kC_ZIHsK...",_OGECa4jFME,"{'kind': 'youtube#comment', 'etag': 'C2r_r0syy...",True,1,True
4,[-1],_OGECa4jFME,"{'kind': 'youtube#comment', 'etag': 'QSzG29jjJ...",True,0,True
...,...,...,...,...,...,...
575,"[{'kind': 'youtube#comment', 'etag': 'bm479ac8...",vbl7QeqfE-Q,"{'kind': 'youtube#comment', 'etag': 'tzua5nozC...",True,6,True
576,"[{'kind': 'youtube#comment', 'etag': 'iBoO9wtg...",vbl7QeqfE-Q,"{'kind': 'youtube#comment', 'etag': 'RYkZTB8QF...",True,3,True
577,"[{'kind': 'youtube#comment', 'etag': '-jZe-WF9...",vbl7QeqfE-Q,"{'kind': 'youtube#comment', 'etag': 'EY-93wV5Q...",True,2,True
578,[-1],vbl7QeqfE-Q,"{'kind': 'youtube#comment', 'etag': 'ushv6S5eu...",True,0,True


In [23]:
dfq = pd.read_csv("../../persist/cleansed/environmental_videos.csv")

dfq_cols = dfq[["videoId", "title"]]

base_df = pd.merge(
    dfq_cols,
    df,
    on="videoId"
).explode(column="comments")

In [24]:
import numpy as np

base_df["comment"] = np.where(base_df["comments"] == -1, base_df["topLevelComment"], base_df["comments"])
base_df["isTopLevel"] = np.where(base_df["comments"] == -1, True, False)
base_df["parentComment"] = np.where(base_df["isTopLevel"], None, base_df["topLevelComment"])
base_df = base_df[[
    "videoId",
    "title",
    "isTopLevel",
    "parentComment",
    "comment",
    "topLevelComment"
]]

base_df

,videoId,title,isTopLevel,parentComment,comment,topLevelComment
0,_OGECa4jFME,Becoming Cousteau | Official Trailer | Nationa...,True,None,"{'kind': 'youtube#comment', 'etag': 'YuWkwE1fD...","{'kind': 'youtube#comment', 'etag': 'YuWkwE1fD..."
1,_OGECa4jFME,Becoming Cousteau | Official Trailer | Nationa...,True,None,"{'kind': 'youtube#comment', 'etag': '6uUdKftsY...","{'kind': 'youtube#comment', 'etag': '6uUdKftsY..."
2,_OGECa4jFME,Becoming Cousteau | Official Trailer | Nationa...,True,None,"{'kind': 'youtube#comment', 'etag': 'F3cSycXbJ...","{'kind': 'youtube#comment', 'etag': 'F3cSycXbJ..."
3,_OGECa4jFME,Becoming Cousteau | Official Trailer | Nationa...,False,"{'kind': 'youtube#comment', 'etag': 'C2r_r0syy...","{'kind': 'youtube#comment', 'etag': 'kC_ZIHsKF...","{'kind': 'youtube#comment', 'etag': 'C2r_r0syy..."
3,_OGECa4jFME,Becoming Cousteau | Official Trailer | Nationa...,True,None,"{'kind': 'youtube#comment', 'etag': 'C2r_r0syy...","{'kind': 'youtube#comment', 'etag': 'C2r_r0syy..."
...,...,...,...,...,...,...
575,-FMvbefokq8,The Real Amazons | Podcast | Overheard at Nati...,True,None,"{'kind': 'youtube#comment', 'etag': 'RKyC6iA4G...","{'kind': 'youtube#comment', 'etag': 'RKyC6iA4G..."
576,-FMvbefokq8,The Real Amazons | Podcast | Overheard at Nati...,True,None,"{'kind': 'youtube#comment', 'etag': '6je52FlJ0...","{'kind': 'youtube#comment', 'etag': '6je52FlJ0..."
577,-FMvbefokq8,The Real Amazons | Podcast | Overheard at Nati...,True,None,"{'kind': 'youtube#comment', 'etag': 'qU7StvOqq...","{'kind': 'youtube#comment', 'etag': 'qU7StvOqq..."
578,-FMvbefokq8,The Real Amazons | Podcast | Overheard at Nati...,True,None,"{'kind': 'youtube#comment', 'etag': 'XO0LFmLdo...","{'kind': 'youtube#comment', 'etag': 'XO0LFmLdo..."


In [25]:
from pandas import json_normalize
normalize_comment_df = json_normalize(base_df["comment"])
base_df[normalize_comment_df.columns] = normalize_comment_df
base_df

,videoId,title,isTopLevel,parentComment,comment,topLevelComment,kind,etag,id,snippet.videoId,...,snippet.authorDisplayName,snippet.authorProfileImageUrl,snippet.authorChannelUrl,snippet.authorChannelId.value,snippet.canRate,snippet.viewerRating,snippet.likeCount,snippet.publishedAt,snippet.updatedAt,snippet.parentId
0,_OGECa4jFME,Becoming Cousteau | Official Trailer | Nationa...,True,None,"{'kind': 'youtube#comment', 'etag': 'YuWkwE1fD...","{'kind': 'youtube#comment', 'etag': 'YuWkwE1fD...",youtube#comment,YuWkwE1fDH8Rwuo_lTx5twChVKY,Ugz2QH5licjtQtc9tDh4AaABAg,_OGECa4jFME,...,Nixon,https://yt3.ggpht.com/ytc/AKedOLSxleNHQYQQC7Hd...,http://www.youtube.com/channel/UCteM6MAM7TvCI6...,UCteM6MAM7TvCI6-dJ_7b6KA,True,none,0,2021-10-23T16:53:41Z,2021-10-23T16:53:41Z,NaN
1,_OGECa4jFME,Becoming Cousteau | Official Trailer | Nationa...,True,None,"{'kind': 'youtube#comment', 'etag': '6uUdKftsY...","{'kind': 'youtube#comment', 'etag': '6uUdKftsY...",youtube#comment,6uUdKftsYXzzk6QTUt97-h8iAcE,UgzHmd28uC_pIky5VhZ4AaABAg,_OGECa4jFME,...,D-.-RAiL Gaming,https://yt3.ggpht.com/ytc/AKedOLQrr3ynLvWdg3A-...,http://www.youtube.com/channel/UCU-Vzvlq_5QN_i...,UCU-Vzvlq_5QN_imM4cyGR6A,True,none,2,2021-10-22T13:38:37Z,2021-10-22T13:38:37Z,NaN
2,_OGECa4jFME,Becoming Cousteau | Official Trailer | Nationa...,True,None,"{'kind': 'youtube#comment', 'etag': 'F3cSycXbJ...","{'kind': 'youtube#comment', 'etag': 'F3cSycXbJ...",youtube#comment,F3cSycXbJ09UvEpKoQ5szxiHVdw,UgzhWP-SVHN2d7QYlpN4AaABAg,_OGECa4jFME,...,bengkel pancing uchuy,https://yt3.ggpht.com/ZxjTXKqEDA-GM4umxOslmpkX...,http://www.youtube.com/channel/UCaC_VwqeFTNXAe...,UCaC_VwqeFTNXAea6m1ezhXw,True,none,1,2021-10-22T12:52:34Z,2021-10-22T12:52:34Z,NaN
3,_OGECa4jFME,Becoming Cousteau | Official Trailer | Nationa...,False,"{'kind': 'youtube#comment', 'etag': 'C2r_r0syy...","{'kind': 'youtube#comment', 'etag': 'kC_ZIHsKF...","{'kind': 'youtube#comment', 'etag': 'C2r_r0syy...",youtube#comment,kC_ZIHsKFGhmmdwnWG3Mkpn2jcI,Ugzx_6T_qaISlyjNTu94AaABAg.9TmpzniJhAO9V97RTZsy4I,_OGECa4jFME,...,Nicolas Mazzola,https://yt3.ggpht.com/ytc/AKedOLRDHQnHA2fIXRU9...,http://www.youtube.com/channel/UCnfd6C39V0gFLa...,UCnfd6C39V0gFLapJG8QeuXQ,True,none,0,2021-11-25T02:09:15Z,2021-11-25T02:09:15Z,Ugzx_6T_qaISlyjNTu94AaABAg
3,_OGECa4jFME,Becoming Cousteau | Official Trailer | Nationa...,True,None,"{'kind': 'youtube#comment', 'etag': 'C2r_r0syy...","{'kind': 'youtube#comment', 'etag': 'C2r_r0syy...",youtube#comment,kC_ZIHsKFGhmmdwnWG3Mkpn2jcI,Ugzx_6T_qaISlyjNTu94AaABAg.9TmpzniJhAO9V97RTZsy4I,_OGECa4jFME,...,Nicolas Mazzola,https://yt3.ggpht.com/ytc/AKedOLRDHQnHA2fIXRU9...,http://www.youtube.com/channel/UCnfd6C39V0gFLa...,UCnfd6C39V0gFLapJG8QeuXQ,True,none,0,2021-11-25T02:09:15Z,2021-11-25T02:09:15Z,Ugzx_6T_qaISlyjNTu94AaABAg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
575,-FMvbefokq8,The Real Amazons | Podcast | Overheard at Nati...,True,None,"{'kind': 'youtube#comment', 'etag': 'RKyC6iA4G...","{'kind': 'youtube#comment', 'etag': 'RKyC6iA4G...",youtube#comment,S-5yi22CtwZ7vs54FIMcikkthiM,UgxKyO5Z9ejqMRrWeIB4AaABAg,vbl7QeqfE-Q,...,Agenzia delle Entrate,https://yt3.ggpht.com/ytc/AKedOLThxN7SoQqH5LQB...,http://www.youtube.com/channel/UCJe2FkSsHz1SeK...,UCJe2FkSsHz1SeKBD_4EoQfQ,True,none,0,2021-06-28T14:56:15Z,2021-06-28T14:56:15Z,NaN
576,-FMvbefokq8,The Real Amazons | Podcast | Overheard at Nati...,True,None,"{'kind': 'youtube#comment', 'etag': '6je52FlJ0...","{'kind': 'youtube#comment', 'etag': '6je52FlJ0...",youtube#comment,2wRH6hRQgi5r9pPOv2YpRUGXdeM,UgwCM92nEvPf-FRctH14AaABAg,vbl7QeqfE-Q,...,Abir Mef,https://yt3.ggpht.com/ytc/AKedOLSQIjViUI18nGnG...,http://www.youtube.com/channel/UCCBn_CtkiwqMGD...,UCCBn_CtkiwqMGDQW9ESTq9w,True,none,1,2021-06-28T06:55:53Z,2021-06-28T06:55:53Z,NaN
577,-FMvbefokq8,The Real Amazons | Podcast | Overheard at Nati...,True,None,"{'kind': 'youtube#comment', 'etag': 'qU7StvOqq...","{'kind': 'youtube#comment', 'etag': 'qU7StvOqq...",youtube#comment,mSJwOwl5IXa

In [27]:
base_df.to_parquet("../../persist/cleansed/comments.parquet", index=False)